## Daily Pivot RDHX

In [4]:
%reset -f
import sys, os
sys.path.append('/home/seyedkazemi/codes/mskhelper/')
import pandas as pd, numpy as np
import logging
import datetime, pytz
import mohsenutils, data_provider, m100_preprocessing_helper
import imp 
imp.reload(mohsenutils)
imp.reload(data_provider)
imp.reload(m100_preprocessing_helper)


pub = 'schneider_pub'

for metric in ['PLC_PLC_Q101.Portata_1_hmi','PLC_PLC_Q101.Portata_2_hmi',
               'PLC_PLC_Q101.Temp_mandata','PLC_PLC_Q101.Temp_ritorno',
               'PLC_PLC_Q101.Posizione_ty141','PLC_PLC_Q101.Posizione_ty142',
               'PLC_PLC_Q101.T_ritorno_hmi','PLC_PLC_Q101.T_mandata_hmi',
               'PLC_PLC_Q101.Portata_1','PLC_PLC_Q101.Portata_2',
               'PLC_PLC_Q101.Delta_temp']:
    
    
    print('__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__')
    logging.info(str('__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'))
    log_file_name = 'CRAC_log_raw_data_cook_'+datetime.datetime.strftime(datetime.datetime.now(pytz.timezone('Europe/Rome')), 
                                                                      '%Y-%m-%d %H:%M:%S').replace(':','-').replace(' ','_')+'.log'
    logging.basicConfig(filename=log_file_name,level=logging.INFO, format='%(levelname)s:%(asctime)s:%(lineno)d:%(message)s')


    path = os.path.join('/home/seyedkazemi/dataset/Marconi100_backup/',pub,metric,'')

    if not os.path.isdir(path+'/'+metric+'_done/'):
        os.makedirs(path+'/'+metric+'_done/')

    if not os.path.isdir(path+'/'+metric+'_pivot/'):
        os.makedirs(path+'/'+metric+'_pivot/')    

    files_list = sorted([file for file in os.listdir(path) if '.csv.gz' in file])
    for file in files_list:
        print(file)
        logging.info(str(file))
        
        data = pd.read_csv(path+file, compression='gzip', index_col='timestamp')
        
        data.loc[data['value'] == 'Unavailable', 'value'] = np.nan
        data.loc[data['value'] == 'unknown', 'value'] = np.nan
        data['value'] = data['value'].astype(float)


        
        print('Data shape: ', data.shape)
        logging.info('Data shape: '+ str(data.shape))


        data = data.pipe(m100_preprocessing_helper.remove_Unnamed, log=True).\
        pipe(m100_preprocessing_helper.unique_val, log=True).\
        pipe(m100_preprocessing_helper.pvt, index='timestamp', freq='10T', columns='panel', values='value', log=True)

        data.to_csv(path+'/'+metric+'_pivot/'+file.split('.')[0]+'.csv.gz', compression='gzip')
        os.replace(path+file, path+metric+'_done/'+file)
        print(50*'#')
        logging.info(str(50*'#'))
    
    
    

__PLC_PLC_Q101.Portata_1_hmi__PLC_PLC_Q101.Portata_1_hmi__PLC_PLC_Q101.Portata_1_hmi__PLC_PLC_Q101.Portata_1_hmi__PLC_PLC_Q101.Portata_1_hmi__PLC_PLC_Q101.Portata_1_hmi__
__PLC_PLC_Q101.Portata_2_hmi__PLC_PLC_Q101.Portata_2_hmi__PLC_PLC_Q101.Portata_2_hmi__PLC_PLC_Q101.Portata_2_hmi__PLC_PLC_Q101.Portata_2_hmi__PLC_PLC_Q101.Portata_2_hmi__
__PLC_PLC_Q101.Temp_mandata__PLC_PLC_Q101.Temp_mandata__PLC_PLC_Q101.Temp_mandata__PLC_PLC_Q101.Temp_mandata__PLC_PLC_Q101.Temp_mandata__PLC_PLC_Q101.Temp_mandata__
__PLC_PLC_Q101.Temp_ritorno__PLC_PLC_Q101.Temp_ritorno__PLC_PLC_Q101.Temp_ritorno__PLC_PLC_Q101.Temp_ritorno__PLC_PLC_Q101.Temp_ritorno__PLC_PLC_Q101.Temp_ritorno__
__PLC_PLC_Q101.Posizione_ty141__PLC_PLC_Q101.Posizione_ty141__PLC_PLC_Q101.Posizione_ty141__PLC_PLC_Q101.Posizione_ty141__PLC_PLC_Q101.Posizione_ty141__PLC_PLC_Q101.Posizione_ty141__
__PLC_PLC_Q101.Posizione_ty142__PLC_PLC_Q101.Posizione_ty142__PLC_PLC_Q101.Posizione_ty142__PLC_PLC_Q101.Posizione_ty142__PLC_PLC_Q101.Posizione_

## Create One Big file

In [6]:
%reset -f
import sys, os
sys.path.append('/home/seyedkazemi/codes/HPCRoomModel/mskhelper/')
import pandas as pd, numpy as np
import glob
import logging
import datetime, pytz
import mohsenutils
import imp 
import m100_preprocessing_helper
imp.reload(m100_preprocessing_helper)



pth = '/home/seyedkazemi/dataset/Marconi100_backup/'

pub = 'schneider_pub'
for metric in ['PLC_PLC_Q101.Portata_1_hmi','PLC_PLC_Q101.Portata_2_hmi',
               'PLC_PLC_Q101.Temp_mandata','PLC_PLC_Q101.Temp_ritorno',
               'PLC_PLC_Q101.Posizione_ty141','PLC_PLC_Q101.Posizione_ty142',
               'PLC_PLC_Q101.T_ritorno_hmi','PLC_PLC_Q101.T_mandata_hmi',
               'PLC_PLC_Q101.Portata_1','PLC_PLC_Q101.Portata_2',
               'PLC_PLC_Q101.Delta_temp']:
    m100_preprocessing_helper.concat_multiple_csvs(csvdir=os.path.join(pth, pub, metric, metric+'_pivot'),
                                                   big_csv_path_name=os.path.join(pth, pub, metric, metric+'_pivot', metric+'_big.csv.gz'),
                                                   old_pivot_path=os.path.join(pth, pub, metric, metric+'_pivot_old_pivot',''),
                                                   freq='10T')

['/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Portata_1_hmi/PLC_PLC_Q101.Portata_1_hmi_pivot/PLC_PLC_Q101.Portata_1_hmi_big.csv.gz']
Number of file 1
Your pivot already has been updated.
['/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Portata_2_hmi/PLC_PLC_Q101.Portata_2_hmi_pivot/PLC_PLC_Q101.Portata_2_hmi_big.csv.gz']
Number of file 1
Your pivot already has been updated.
['/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Temp_mandata/PLC_PLC_Q101.Temp_mandata_pivot/PLC_PLC_Q101.Temp_mandata_big.csv.gz']
Number of file 1
Your pivot already has been updated.
['/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Temp_ritorno/PLC_PLC_Q101.Temp_ritorno_pivot/PLC_PLC_Q101.Temp_ritorno_big.csv.gz']
Number of file 1
Your pivot already has been updated.
['/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Posizione_ty141/PLC_PLC_Q101.Posizione_ty141_pivot/PLC_PLC_Q101.Posizione_ty1

## RDHX/Pumps DATA ONE csv file

In [8]:
%reset -f
import sys, os
sys.path.append('/home/seyedkazemi/codes/HPCRoomModel/mskhelper/')
import pandas as pd, numpy as np
import dask.dataframe as ds
import logging
import datetime, pytz
import mohsenutils
import imp 
import m100_preprocessing_helper
imp.reload(m100_preprocessing_helper)

kwargs = {'compression':'gzip', 'index_col':'timestamp', 'date_parser':pd.to_datetime, 'parse_dates':True}

Portata_1_hmi = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Portata_1_hmi/PLC_PLC_Q101.Portata_1_hmi_pivot/PLC_PLC_Q101.Portata_1_hmi_big.csv.gz', **kwargs)
Portata_2_hmi = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Portata_2_hmi/PLC_PLC_Q101.Portata_2_hmi_pivot/PLC_PLC_Q101.Portata_2_hmi_big.csv.gz', **kwargs)

Portata_1 = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Portata_1/PLC_PLC_Q101.Portata_1_pivot/PLC_PLC_Q101.Portata_1_big.csv.gz', **kwargs)
Portata_2 = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Portata_2/PLC_PLC_Q101.Portata_2_pivot/PLC_PLC_Q101.Portata_2_big.csv.gz', **kwargs)

Temp_mandata = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Temp_mandata/PLC_PLC_Q101.Temp_mandata_pivot/PLC_PLC_Q101.Temp_mandata_big.csv.gz', **kwargs)
Temp_ritorno = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Temp_ritorno/PLC_PLC_Q101.Temp_ritorno_pivot/PLC_PLC_Q101.Temp_ritorno_big.csv.gz', **kwargs)

Posizione_ty141 = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Posizione_ty141/PLC_PLC_Q101.Posizione_ty141_pivot/PLC_PLC_Q101.Posizione_ty141_big.csv.gz', **kwargs)
Posizione_ty142 = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Posizione_ty142/PLC_PLC_Q101.Posizione_ty142_pivot/PLC_PLC_Q101.Posizione_ty142_big.csv.gz', **kwargs)

T_ritorno_hmi = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.T_ritorno_hmi/PLC_PLC_Q101.T_ritorno_hmi_pivot/PLC_PLC_Q101.T_ritorno_hmi_big.csv.gz', **kwargs)
T_mandata_hmi = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.T_mandata_hmi/PLC_PLC_Q101.T_mandata_hmi_pivot/PLC_PLC_Q101.T_mandata_hmi_big.csv.gz', **kwargs)

Delta_temp = pd.read_csv('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/PLC_PLC_Q101.Delta_temp/PLC_PLC_Q101.Delta_temp_pivot/PLC_PLC_Q101.Delta_temp_big.csv.gz', **kwargs)


Portata_1_hmi.rename(columns={'Q101':'Portata_1_hmi_Q101','Q102':'Portata_1_hmi_Q102'}, inplace=True)
Portata_2_hmi.rename(columns={'Q101':'Portata_2_hmi_Q101','Q102':'Portata_2_hmi_Q102'}, inplace=True)

Portata_1.rename(columns={'Q101':'Portata_1_Q101','Q102':'Portata_1_Q102'}, inplace=True)
Portata_2.rename(columns={'Q101':'Portata_2_Q101','Q102':'Portata_2_Q102'}, inplace=True)

Temp_mandata.rename(columns={'Q101':'Temp_mandata_Q101','Q102':'Temp_mandata_Q102'}, inplace=True)
Temp_ritorno.rename(columns={'Q101':'Temp_ritorno_Q101','Q102':'Temp_ritorno_Q102'}, inplace=True)

Posizione_ty141.rename(columns={'Q101':'Posizione_ty141_Q101','Q102':'Posizione_ty141_Q102'}, inplace=True)
Posizione_ty142.rename(columns={'Q101':'Posizione_ty142_Q101','Q102':'Posizione_ty142_Q102'}, inplace=True)

T_ritorno_hmi.rename(columns={'Q101':'T_ritorno_hmi_Q101','Q102':'T_ritorno_hmi_Q102'}, inplace=True)
T_mandata_hmi.rename(columns={'Q101':'T_mandata_hmi_Q101','Q102':'T_mandata_hmi_Q102'}, inplace=True)

Delta_temp.rename(columns={'Q101':'Delta_temp_Q101','Q102':'Delta_temp_Q102'}, inplace=True)


Portata_1_hmi, Portata_2_hmi = mohsenutils.same_index_range(Portata_1_hmi, Portata_2_hmi)
Temp_mandata, Temp_ritorno = mohsenutils.same_index_range(Temp_mandata, Temp_ritorno)
Posizione_ty141, Posizione_ty142 = mohsenutils.same_index_range(Posizione_ty141, Posizione_ty142)
T_ritorno_hmi, T_mandata_hmi = mohsenutils.same_index_range(T_ritorno_hmi, T_mandata_hmi)


Portata_1_hmi, Portata_2_hmi = mohsenutils.same_index_range(Portata_1_hmi, Portata_2_hmi)
Portata_1_hmi, Portata_1  = mohsenutils.same_index_range(Portata_1_hmi, Portata_1)
Portata_1_hmi, Portata_2  = mohsenutils.same_index_range(Portata_1_hmi, Portata_2)
Portata_1_hmi, Temp_mandata = mohsenutils.same_index_range(Portata_1_hmi, Temp_mandata)
Portata_1_hmi, Temp_ritorno = mohsenutils.same_index_range(Portata_1_hmi, Temp_ritorno)
Portata_1_hmi, Posizione_ty141 = mohsenutils.same_index_range(Portata_1_hmi, Posizione_ty141)
Portata_1_hmi, Posizione_ty142 = mohsenutils.same_index_range(Portata_1_hmi, Posizione_ty142)
Portata_1_hmi, T_ritorno_hmi = mohsenutils.same_index_range(Portata_1_hmi, T_ritorno_hmi)
Portata_1_hmi, T_mandata_hmi = mohsenutils.same_index_range(Portata_1_hmi, T_mandata_hmi)
Portata_1_hmi, Delta_temp = mohsenutils.same_index_range(Portata_1_hmi, Delta_temp)

Portata_1_hmi, Portata_2_hmi = mohsenutils.same_index_range(Portata_1_hmi, Portata_2_hmi)
Portata_1_hmi, Portata_1  = mohsenutils.same_index_range(Portata_1_hmi, Portata_1)
Portata_1_hmi, Portata_2  = mohsenutils.same_index_range(Portata_1_hmi, Portata_2)
Portata_1_hmi, Temp_mandata = mohsenutils.same_index_range(Portata_1_hmi, Temp_mandata)
Portata_1_hmi, Temp_ritorno = mohsenutils.same_index_range(Portata_1_hmi, Temp_ritorno)
Portata_1_hmi, Posizione_ty141 = mohsenutils.same_index_range(Portata_1_hmi, Posizione_ty141)
Portata_1_hmi, Posizione_ty142 = mohsenutils.same_index_range(Portata_1_hmi, Posizione_ty142)
Portata_1_hmi, T_ritorno_hmi = mohsenutils.same_index_range(Portata_1_hmi, T_ritorno_hmi)
Portata_1_hmi, T_mandata_hmi = mohsenutils.same_index_range(Portata_1_hmi, T_mandata_hmi)
Portata_1_hmi, Delta_temp = mohsenutils.same_index_range(Portata_1_hmi, Delta_temp)

data_RDHX = mohsenutils.mrg(None, 'outer', Portata_1_hmi, Portata_2_hmi, Portata_1, Portata_2, T_ritorno_hmi, T_mandata_hmi, Temp_mandata, Temp_ritorno,  Posizione_ty141, Posizione_ty142, Delta_temp)


data_RDHX.index.name = 'timestamp'
data_RDHX.to_csv('/home/seyedkazemi/dataset/M100_Complete/RDHX_big.csv.gz', compression='gzip')

display(data_RDHX)

2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00
2021-03-29 16:30:00 2022-04-07 00:00:00


,Portata_1_hmi_Q101,Portata_1_hmi_Q102,Portata_2_hmi_Q101,Portata_2_hmi_Q102,Portata_1_Q101,Portata_1_Q102,Portata_2_Q101,Portata_2_Q102,T_ritorno_hmi_Q101,T_ritorno_hmi_Q102,...,Temp_mandata_Q101,Temp_mandata_Q102,Temp_ritorno_Q101,Temp_ritorno_Q102,Posizione_ty141_Q101,Posizione_ty141_Q102,Posizione_ty142_Q101,Posizione_ty142_Q102,Delta_temp_Q101,Delta_temp_Q102
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-03-29 16:30:00,939.238095,939.454545,937.714286,938.272727,4696.428571,4696.181818,4692.047619,4690.545455,239.952381,244.954545,...,179.952381,182.409091,239.952381,245.000000,79.000000,79.000000,76.666667,76.772727,60.0,60.0
2021-03-29 16:40:00,933.161290,933.161290,931.903226,932.548387,4663.193548,4665.258065,4659.870968,4662.806452,239.967742,245.064516,...,180.000000,182.774194,239.967742,245.064516,79.000000,79.000000,76.516129,76.741935,60.0,60.0
2021-03-29 16:50:00,934.600000,935.066667,929.966667,930.366667,4674.566667,4674.433333,4649.600000,4652.766667,240.000000,245.000000,...,180.033333,182.900000,240.000000,245.000000,79.000000,79.000000,76.633333,76.900000,60.0,60.0
2021-03-29 17:00:00,933.800000,934.433333,933.033333,933.433333,4671.966667,4671.233333,4664.733333,4667.600000,240.033333,245.100000,...,179.966667,182.700000,240.033333,245.100000,79.000000,79.000000,76.866667,76.866667,60.0,60.0
2021-03-29 17:10:00,950.566667,950.833333,947.500000,947.700000,4753.333333,4754.700000,4737.433333,4738.166667,240.433333,245.466667,...,180.100000,182.866667,240.400000,245.433333,79.433333,79.433333,78.533333,78.600000,60.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-06 23:20:00,0.500000,0.666667,0.000000,0.000000,2.666667,2.833333,0.000000,0.000000,215.066667,220.000000,...,157.833333,160.000000,215.166667,220.000000,79.000000,79.000000,76.000000,77.000000,60.0,60.0
2022-04-06 23:30:00,0.766667,0.800000,0.000000,0.000000,2.700000,2.833333,0.000000,0.000000,215.133333,219.933333,...,157.966667,160.033333,215.166667,219.933333,79.000000,79.000000,76.300000,77.000000,60.0,60.0
2022-04-06 23:40:00,0.633333,0.766667,0.000000,0.000000,2.566667,2.800000,0.000000,0.000000,215.300000,220.100000,...,157.933333,160.366667,215.266667,220.100000,79.200000,79.200000,77.033333,77.433333,60.0,60.0


## To Check

In [ ]:
%reset -f
import sys, os
sys.path.append('/home/seyedkazemi/codes/HPCRoomModel/mskhelper/')
import pandas as pd, numpy as np
import logging
import datetime, pytz
import mohsenutils, data_provider, m100_preprocessing_helper
import imp 
import dask.dataframe as ds

imp.reload(mohsenutils)
imp.reload(data_provider)
imp.reload(m100_preprocessing_helper)



path = '/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/'
metric = 'PLC_PLC_Q101.T_mandata_hmi'
# display(os.listdir(path))


data = ds.read_csv(path+'/'+metric+'/'+'*.csv.gz', compression='gzip',dtype={'value': 'object'})
data = data.compute()
data.set_index('timestamp', inplace=True)

display(data)
# display(data['value'].unique())

data.loc[data['value'] == 'unknown', 'value'] = np.nan
data.loc[data['value'] == 'Unavailable', 'value'] = np.nan
data['value'] = data['value'].astype(float)
display(data)

print(data.value.sum())

data.pipe(m100_preprocessing_helper.remove_Unnamed, log=False).\
pipe(m100_preprocessing_helper.unique_val, log=False).\
pipe(m100_preprocessing_helper.pvt, index='timestamp', freq='10T', columns='panel', values='value', log=False)






In [ ]:
%reset -f
import sys, os
sys.path.append('/home/seyedkazemi/codes/HPCRoomModel/mskhelper/')
import pandas as pd, numpy as np
import logging
import datetime, pytz
import mohsenutils, data_provider, m100_preprocessing_helper
import imp 
import dask.dataframe as ds

imp.reload(mohsenutils)
imp.reload(data_provider)
imp.reload(m100_preprocessing_helper)

'PLC_PLC_Q101.Portata_1_hmi'
'PLC_PLC_Q101.Portata_2_hmi'

'PLC_PLC_Q101.Temp_mandata'
'PLC_PLC_Q101.Temp_ritorno'

'PLC_PLC_Q101.Posizione_ty141'
'PLC_PLC_Q101.Posizione_ty142'

'PLC_PLC_Q101.T_mandata_hmi'


path = '/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/'
metric_list = os.listdir(path)
# for metric in metric_list:
for metric in ['PLC_PLC_Q101.Portata_attiva']:
    
    
    print(4*('+++'+metric+'+++'+metric+'+++'))
    print()
    data = ds.read_csv(path+'/'+metric+'/'+'*.csv.gz', compression='gzip',dtype={'value': 'object'})
    data = data.compute()
    data.set_index('timestamp', inplace=True)

#     display(data)
    data = data.pipe(m100_preprocessing_helper.remove_Unnamed)
    display(data)
    data = data.pipe(m100_preprocessing_helper.unique_val)
    print('LEN UNIQUE VALUE: ',len(data.value.unique()))
    print()
    print()
    
    

In [ ]:
os.listdir(path)


data = pd.read_csv(path+'Ext_Air_Sensor_C_Temperature_2021-04-28_00-00-00_to_2021-04-29_00-00-00.csv.gz', compression='gzip')
data.set_index('timestamp')

display(data)
data.loc[data['value'] == 'Unavailable', 'value'] = np.nan
data['value'] = data['value'].astype(float)
display(data, data.info())

# print(data.value.sum())

data.pipe(m100_preprocessing_helper.remove_Unnamed, log=False).\
pipe(m100_preprocessing_helper.unique_val, log=False).\
pipe(m100_preprocessing_helper.pvt, index='timestamp', freq='10T', columns='device', values='value', log=False)

In [ ]:
os.listdir('/home/seyedkazemi/dataset/Marconi100_backup/schneider_pub/')